In [20]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]
scr = None
grayG = None
hsv=None

class camshift:
#     r,h,c,w = 0,10,360,10
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    trackcount = 0
    cnt = 10
    has = True
    global hsv, loadModel, scr, grayG
    def  __init__(self,cx, cy):
        try:
            x, y, h, w = (cx-25), (cy-25), 50, 50 
            self.roi = hsv[y:y+w, x:x+h,:]
#             mean = np.mean(self.roi)
# #             self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2GRAY)
#             mask = cv2.inRange(self.roi,np.array([mean-5]), np.array([mean+5]))
            self.roi_hist = cv2.calcHist([self.roi], [0,1], None, [180,255], [0,180,0,255])
            cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
            self.track_window = (x,y,w,h)
            self.cnt=random.randint(6,12)
            self.has = True
        except:
            self.has = False
    def comp(self):
        if self.trackcount<=self.cnt:
            return False #remain
        else:
            return True #delete
    def track_csft(self):
        try:       
            dst = cv2.calcBackProject([hsv],[0,1],self.roi_hist,[0,180,0,255],1)
            self.ret, self.track_window = cv2.meanShift(dst, self.track_window, self.term_crit)
            self.trackcount += 1
        except:
            self.trackcount += 1
    def preTrackPred_csft(self):
        loadModel = None
    def coord(self):
        x,y,w,h =  self.track_window
        return [[x,y],[x+w,y],[x+w,y+h],[x,y+h]]
        
        
    
def sift_knn(img1,img2,part1,part2):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.7 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
            infolist.append(info)
#             cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



#     draw_params = dict(matchColor = (255, 0,0),
#                        singlePointColor = (0,0,255),
#                        matchesMask = matchesMask,
#                        flags = 2)
#     res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],40)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
        
    for inx in range(len(centroid)):
        cv2.circle(img2,(int(centroid[inx][0]/10),int(centroid[inx][1]/10)),5,(0,0,255),-1)
#         centroid[inx][0]+=part1
#         centroid[inx][1]+=part2
    centroidsAll += list(np.array(centroid)+[10*part2,10*part1,0,0])
    
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll, grayG
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*8
    prev1 = []
    shape = [26*15,68*15]
    cfstObj = []
    global loaded_model, hsv, scr
    
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
##    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(scr, cv2.COLOR_BGR2HSV)
        grayG = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
#         cv2.imshow('hsv0', hsv[:,:,0:2])
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,0,shape[1]//3))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,0,2*shape[1]//3))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,shape[0]//2,0))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,shape[0]//2,shape[1]//3))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,shape[0]//2,2*shape[1]//3))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        
#         hsvR = hsv[shape[0]-40:shape[0],200:shape[1]-200,:]
#         roihist = cv2.calcHist([hsvR],[0, 1], None, [180, 256], [0, 180, 0, 256] )
#         cv2.normalize(roihist,roihist,0,255,cv2.NORM_MINMAX)
#         dst = cv2.calcBackProject([hsv],[0,1],roihist,[0,180,0,256],1)
#         disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
#         cv2.filter2D(dst,-1,disc,dst)
#         ret,thresh = cv2.threshold(dst,50,255,0)
#         thresh = cv2.merge((thresh,thresh,thresh))
#         res = cv2.bitwise_and(scr,thresh)
#         cv2.imshow('resss',res)
        
        
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
#         print("no",len(cfstObj))
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            x,y,w,h = obj.track_window
            cv2.rectangle(scr, (x,y), (x+w,y+h), [0,0,255],2)
            if obj.comp():
                cfstObj.remove(obj)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            has=1
            for obj in cfstObj:
#                 pts = cv2.boxPoints(obj.ret)
                polygon = Polygon(obj.coord())
                if polygon.contains(point):
                    has=0
                    break
            if has:        
                tempobj = camshift(int(cnt[0]/10),int(cnt[1]/10))
                retCt = tempobj.has
                if retCt:  
                    tempobj.track_csft()
                    x,y,w,h = tempobj.track_window        
                    cv2.rectangle(scr, (x,y), (x+w,y+h), [225,0,255],2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(scr,str(x)+str(y),(x,y), font, 1,(255,255,0),1,cv2.LINE_AA)
#                     tempobj.preTrackPred_csft()
                    cfstObj.append(tempobj)
            
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny) 
        cv2.imshow('print3',scr)   
 

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
Model loaded from disk
running
FPS:  0.4004837921328494


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:180: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
Exception in thread Thread-7502:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-20-6e745baf1ec6>", line 114, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/

FPS:  1.6549220696311062
FPS:  0.4353286456728366
FPS:  0.9393889442338151
FPS:  0.347008365985386


Exception in thread Thread-7526:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-20-6e745baf1ec6>", line 114, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  0.7249176619591338
FPS:  0.3346179369339571
FPS:  0.5733303953533259
FPS:  0.36700655917613084
FPS:  0.5209177274183194
FPS:  0.3470481326829204
FPS:  0.48510537207742016
FPS:  0.30194301436644977
FPS:  0.4572745316982538
FPS:  0.3059490597453177
FPS:  0.5129936747596541
FPS:  0.33944431796910146
FPS:  0.549150874486223
FPS:  0.3806439604286027
FPS:  0.40037957891406867
FPS:  0.3118665159346737
FPS:  0.4369554145050974
FPS:  0.34167520317570355
FPS:  0.3991893446060142


In [69]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]
scr = None
grayG = None
hsv=None

class camshift:
#     r,h,c,w = 0,10,360,10
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    trackcount = 0
    cnt = 10
    has = True
    global hsv, loadModel, scr, grayG
    def  __init__(self,cx, cy):
        try:
            r, h, c, w = (cx-20), 40, (cy-20), 40 
            self.roi = hsv[r:r+h, c:c+w,:]
#             mean = np.mean(self.roi)
# #             self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2GRAY)
#             mask = cv2.inRange(self.roi,np.array([mean-5]), np.array([mean+5]))
            self.roi_hist = cv2.calcHist([self.roi], [0,1], None, [180,255], [0,180,0,255])
            cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
            self.track_window = (c,r,w,h)
            self.cnt=random.randint(10,100)
            self.has = True
        except:
            self.has = False
    def comp(self):
#         pts[1][1]-pts[0][1]
#         pts[1][0]-pts[0][0]
#         pts[2][1]-pts[1][1]
#         pts[2][0]-pts[1][0]
        if self.trackcount<=self.cnt:
            return False#remain
        else:
            return True#delete
    def track_csft(self):
        try:       
            dst = cv2.calcBackProject([hsv],[0,1],self.roi_hist,[0,180,0,255],1)
            self.ret, self.track_window = cv2.CamShift(dst, self.track_window, self.term_crit)
            self.trackcount += 1
        except:
            self.trackcount += 1
#         print(self.ret)
    def preTrackPred_csft(self):
        loadModel = None
    def plot(self):
        pts = cv2.boxPoints(self.ret)
        pts = np.int0(pts)
        return [pts]
#         img2 = cv2.polylines(frame,[pts],True, 255,2)
#         cv2.imshow('img2',img2)        
        
    
def sift_knn(img1,img2,part):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.9 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 2)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],10)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
    centroidsAll += list(centroid)
    for cnt in centroid:
        cv2.circle(img2,(int(cnt[0]/10),int(cnt[1]/10)),5,(0,0,255),-1)
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll, grayG
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*8
    prev1 = []
    shape = [260*2,680*2]
    cfstObj = []
    global loaded_model, hsv, scr
    
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
##    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(scr, cv2.COLOR_BGR2HSV)
        grayG = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
        cv2.imshow('hsv0',grayG)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,1))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,2))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,3))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,4))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,5))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
        print("no",len(cfstObj))
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            pts = obj.plot()
            cv2.polylines(scr,pts,True, (0,0,255),1)  
            if obj.comp():
                cfstObj.remove(obj)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            has=1
            for obj in cfstObj:
                pts = cv2.boxPoints(obj.ret)
                polygon = Polygon(np.int0(pts))
                if polygon.contains(point):
                    has=0
                    break
            if has:    
                tempobj = camshift(int(cnt[0]/10),int(cnt[1]/10))
                retCt = tempobj.has
#                 print(int(cnt[0]/10),int(cnt[1]/10))
                if retCt:
                    tempobj.track_csft()
                    tempobj.preTrackPred_csft()
                    cfstObj.append(tempobj)
            
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny) 
        cv2.imshow('print3',scr)   
#         for  

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
Model loaded from disk
running
no 0
FPS:  0.6824159809783418
no 49


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:185: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  0.8036389136907613
no 95
FPS:  0.7505210231755332
no 138
FPS:  0.7483886899793485
no 173
FPS:  0.7080479499749905
no 204
FPS:  0.7002270481074873
no 228
FPS:  0.7781035203385859
no 235
FPS:  0.6918787666843446
no 237
FPS:  0.7966362128296618
no 237
FPS:  0.8074100781883601
no 249
FPS:  0.7893979159824553
no 260
FPS:  0.7609972369129846
no 272
FPS:  0.7111535519155332
no 284
FPS:  0.7367411141655176
no 292
FPS:  0.7188535237294448
no 298
FPS:  0.7730850252635239
no 304
FPS:  0.7275896428165011
no 307
FPS:  0.8281347980950726
no 304
FPS:  0.8278474486817179
no 301
FPS:  0.7883581066717441
no 300
FPS:  0.7341605489596558


In [26]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]
scr = None
grayG = None
hsv=None

class camshift:
#     r,h,c,w = 0,10,360,10
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    trackcount = 0
    cnt = 10
    has = True
    global hsv, loadModel, scr, grayG
    def  __init__(self,cx, cy):
        try:
            x, y, h, w = (cx-10), (cy-10), 20, 20 
            self.roi = hsv[y:y+w, x:x+h,:]
#             mean = np.mean(self.roi)
# #             self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2GRAY)
#             mask = cv2.inRange(self.roi,np.array([mean-5]), np.array([mean+5]))
            self.roi_hist = cv2.calcHist([self.roi], [0,1], None, [180,255], [0,180,0,255])
            cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
            self.track_window = (x,y,w,h)
            self.cnt=random.randint(6,12)
            self.has = True
        except:
            self.has = False
    def comp(self):
#         pts[1][1]-pts[0][1]
#         pts[1][0]-pts[0][0]
#         pts[2][1]-pts[1][1]
#         pts[2][0]-pts[1][0]
        if self.trackcount<=self.cnt:
            return False#remain
        else:
            return True#delete
    def track_csft(self):
        try:       
            dst = cv2.calcBackProject([hsv],[0,1],self.roi_hist,[0,180,0,255],1)
            self.ret, self.track_window = cv2.CamShift(dst, self.track_window, self.term_crit)
            self.trackcount += 1
        except:
            self.trackcount += 1
#         print(self.ret)
    def preTrackPred_csft(self):
        loadModel = None
    def plot(self):
        pts = cv2.boxPoints(self.ret)
        pts = np.int0(pts)
        return [pts]
#         img2 = cv2.polylines(frame,[pts],True, 255,2)
#         cv2.imshow('img2',img2)        
        
    
def sift_knn(img1,img2,part1,part2):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.5 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 2)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],10)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
        
    for inx in range(len(centroid)):
        cv2.circle(img2,(int(centroid[inx][0]/10),int(centroid[inx][1]/10)),5,(0,0,255),-1)
#         centroid[inx][0]+=part1
#         centroid[inx][1]+=part2
    centroidsAll += list(np.array(centroid)+[10*part2,10*part1,0,0])
    
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll, grayG
    vrec = cv2.VideoCapture('v5.mp4',0)
    k = 300*8
    prev1 = []
    shape = [26*15,68*15]
    cfstObj = []
    global loaded_model, hsv, scr
    
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
#     while k:
#         ret, scr = vrec.read()
#         k-=1
#     print("running")
##    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(scr, cv2.COLOR_BGR2HSV)
        grayG = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
        cv2.imshow('hsv0',grayG)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,0,shape[1]//3))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,0,2*shape[1]//3))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,shape[0]//2,0))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,shape[0]//2,shape[1]//3))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,shape[0]//2,2*shape[1]//3))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
#         print("no",len(cfstObj))
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            pts = obj.plot()
            cv2.polylines(scr,pts,True, (0,0,255),2) 
            if obj.comp():
                cfstObj.remove(obj)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            has=1
            for obj in cfstObj:
                pts = cv2.boxPoints(obj.ret)
                polygon = Polygon(pts)
                if polygon.contains(point):
                    has=0
                    break
            if has:    
                tempobj = camshift(int(cnt[0]/10),int(cnt[1]/10))
                retCt = tempobj.has
                if retCt:  
                    tempobj.track_csft()
#                     x,y,w,h = tempobj.track_window        
#                     cv2.rectangle(scr, (x,y), (x+w,y+h), [225,0,255],2)
#                     font = cv2.FONT_HERSHEY_SIMPLEX
#                     cv2.putText(scr,str(x)+str(y),(x,y), font, 1,(255,255,0),1,cv2.LINE_AA)

#                     tempobj.preTrackPred_csft()
                    cfstObj.append(tempobj)
            
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny) 
        cv2.imshow('print3',scr)   
 

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
Model loaded from disk
FPS:  0.6583286507595187


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:189: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
Exception in thread Thread-8180:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-a59242efe0ac>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/

FPS:  1.6122767087491385
FPS:  1.2401350034815286


Exception in thread Thread-8192:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-a59242efe0ac>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  1.5270650291774253
FPS:  1.3897517050891577


...

FPS:  1.4299974259184443
FPS:  1.4625138560877695
FPS:  1.2554026440562909
FPS:  1.3982882378611274
FPS:  1.3786259441621362
FPS:  1.3024037450864527
FPS:  1.325005212446691
FPS:  1.2583139754829562
FPS:  1.256620523014652


Exception in thread Thread-8810:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-a59242efe0ac>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  1.4147171866910193
FPS:  1.4660913182155808
FPS:  1.4471530689911445
FPS:  1.5196456301217296
FPS:  1.5098672428273006
FPS:  1.398525086434523
FPS:  1.5533763116541814
FPS:  1.556884911708678


IllegalArgumentException: Points of LinearRing do not form a closed linestring
Shell is not a LinearRing


ValueError: Null geometry supports no operations